   ##  Description

In this question , I am implemtning EM algorithm for a mixture of  multivariate random variable(x and y)with three clusters 

## Step1: Import important libraries

In [543]:
import numpy as np
import pandas as pd
import random as rand
import matplotlib.pyplot as plt
from scipy.stats import norm
import sys

rand.seed(24)


## Step 2: Initialize the parameters
We have three Parameters Sigma, mu, amd lambda

In [544]:
mu1 = [0, -2]
sig1 = [ [2, 0], [0, 2] ]
mu2 = [1, -3]
sig2 = [ [2, 0], [0, 1] ]
mu3 = [3, 2]
sig3 = [ [1, 0], [0, 2] ]

## Step 3: Gerating samples and concancatating them to form a mixture of Gausian
1. I used random function three samples using parameters we initialized from step 2
2. I Used concatenate function to form a mixture of threee clusters

In [545]:
x1, y1 = np.random.multivariate_normal(mu1, sig1, 10000).T
x2, y2 = np.random.multivariate_normal(mu2, sig2, 10000).T
x3, y3 = np.random.multivariate_normal(mu3, sig3, 10000).T    
xs = np.concatenate((x1, x2, x3))
ys = np.concatenate((y1, y2, y3))
labels = ([1] * 100) + ([2] * 100) +([3] * 100)

# Now I have my data set with axis x and why
data = {'x': xs, 'y': ys, 'label': labels}



## Step 4 : Cluster probability calculation 

1. we start wwith initial random values for our parameters that need to be updated later on
2. Then I calculateed the probability that a given random value comes from a given clusters
 Note: Our covariance is diagnonal covariance


In [546]:
# Initial guess for our cluster parameters
initial_guess = { 'mu1': [1,1],
          'sig1': [ [1, 0], [0, 1] ],
          'mu2': [4,4],
          'sig2': [ [1, 0], [0, 1] ],
          'mu3': [3,3],
          'sig3': [ [1, 0], [0, 1] ],
          'lambda': [0.3, 0.4, 0.3]
        }

def prob(val, mu, sig, lam):
  p = lam
  for i in range(len(val)-1):
    p *= norm.pdf(val[i], mu[i], sig[i][i]) # pdf a given sample point
  return p



## Step 5 : E-Step
 With initial parameters we determine from which cluster a random variable mostly likely came by comparing their
 posterior probability and assign a label corrresponding to cluster number


In [547]:
def expectation(data_set, parameters):
  for i in range(data_set.shape[0]):
    x =data_set['x'][i]
    y = data_set['y'][i]
    #Claculating posterior probability
    p_cluster1 = prob([x, y], list(parameters['mu1']), list(parameters['sig1']), parameters['lambda'][0] )
    p_cluster2 = prob([x, y], list(parameters['mu2']), list(parameters['sig2']), parameters['lambda'][1] )
    p_cluster3 = prob([x, y], list(parameters['mu3']), list(parameters['sig3']), parameters['lambda'][2] )
    
    # clusterig our data in three clusters depending on posterior prob
    if (p_cluster1 >= p_cluster2) & (p_cluster1 >= p_cluster3):
      data_set['label'][i] = 1
    elif  (p_cluster2 >= p_cluster1) & (p_cluster2 >= p_cluster3):
     data_set['label'][i] = 2
    elif  (p_cluster3 >= p_cluster1) & (p_cluster3 >= p_cluster2):
      data_set['label'][i] = 3
    else: data_set['label'][i] = 3
  return data_set  
         

## Step 6: Maximization  Step
1. With parameter and model we find wether the parameter maximizes the likelihood of being sampled correctly from
the gaussian distribution 
2. I found parameters that maximize a given the current lower bound 


In [548]:
def maximization(data_set, parameters):

  samples_in_cluster1 = data_set[data_set['label'] == 1]
  samples_in_cluster2 = data_set[data_set['label'] == 2]
  samples_in_cluster3 = data_set[data_set['label'] == 3]
  #print points_assigned_to_cluster3
  prob_cluster1 = len(samples_in_cluster1) / float(len(data_set))
  prob_cluster2 = len(samples_in_cluster2) / float(len(data_set))
  prob_cluster3 = 1 - prob_cluster1 - prob_cluster2
  parameters['lambda'] = [prob_cluster1, prob_cluster1, prob_cluster1 ]
  parameters['mu1'] = [ samples_in_cluster1['x'].mean(), samples_in_cluster1['y'].mean(), None]
  parameters['mu2'] = [ samples_in_cluster2['x'].mean(), samples_in_cluster2['y'].mean(),None]
  parameters['mu3'] = [ samples_in_cluster3['x'].mean(), samples_in_cluster3['y'].mean(), None]
  parameters['sig1'] = [ [ samples_in_cluster1['x'].std(), 0 ], [ 0,  samples_in_cluster1['y'].std() ], None]
  parameters['sig2'] = [ [ samples_in_cluster2['x'].std(), 0 ], [ 0,  samples_in_cluster2['y'].std() ], None]
  parameters['sig3'] = [ [ samples_in_cluster3['x'].std(), 0 ], [ 0,  samples_in_cluster3['y'].std() ], None]
  return parameters

## Step 7: Convergence
1. We set the minimum threshold of convergence epsilon= 0.07  
2. We calculate the distance( error) between old and neew parameters
3. We compare distnce with epsilon
4. We repeat E-step and M-STEP Ttill distance <=epsilon 
5. New parameters  after each iteration will be printed
## Final check: All iterations, shift(error), and associated error are printed
## Parameters for distribution appears at the last iteration

In [549]:


def distance(old_params, new_params):
  dist = 0
  for param in ['mu1', 'mu2', 'mu3']:
    for i in range(len(old_params)-1):
      dist += (old_params[param][i] - new_params[param][i]) ** 2
  return dist ** 0.5

# loop until parameters converges
shift = sys.maxsize

epsilon = 0.08
iters = 0
df_copy = df
# randomly assign points to their initial clusters
df_copy['label'] = map(lambda x: x+1, np.random.choice(3, len(df)))

#params = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in guess.iteritems()])) 
params = pd.DataFrame.from_dict(initial_guess, orient = 'index')
params = params.transpose()

while shift > epsilon:
  iters += 1
  # E-step
  updated_labels = expectation(df_copy.copy(), params)
#   print (updated_labels)

  # M-step
  updated_parameters = maximization(updated_labels, params.copy())
#Shift= error between old and new parameter
  shift = distance(params, updated_parameters)

  # Printing the mean shift output
  print("iteration_new {}, shift_new {}".format(iters, shift))
  print( params)
  # update labels and params for the next iteration
  df_copy = updated_labels
  params = updated_parameters

    

    

C:\Users\fhvgmana\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\fhvgmana\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\fhvgmana\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

iteration_new 1, shift_new 5.1438446479808775
   mu1    sig1  mu2    sig2  mu3    sig3 lambda
0    1  [1, 0]    4  [1, 0]    3  [1, 0]    0.3
1    1  [0, 1]    4  [0, 1]    3  [0, 1]    0.4
2  NaN     NaN  NaN     NaN  NaN     NaN    0.3
iteration_new 2, shift_new 0.15516950505334753
        mu1                     sig1       mu2                     sig2  \
0  0.137684  [1.2914482350888905, 0]  3.951632  [0.6035982773013226, 0]   
1 -1.857430  [0, 2.1698105556809697]  1.446248  [0, 1.9934235755478706]   
2       NaN                     None       NaN                     None   

        mu3                      sig3    lambda  
0  2.656476  [0.38895567537066256, 0]  0.613333  
1 -0.302824    [0, 2.991533891626232]  0.613333  
2       NaN                      None  0.613333  
iteration_new 3, shift_new 0.10415977706202975
        mu1                     sig1       mu2                     sig2  \
0  0.086919  [1.2725201369196326, 0]  3.983965  [0.5964601793451143, 0]   
1 -1.923776   [0,

iteration_new 17, shift_new 0.12258968487968183
        mu1                     sig1       mu2                     sig2  \
0 -0.387511  [1.1464340344627701, 0]  3.512205  [0.6749937030997989, 0]   
1 -2.176124  [0, 1.7348920770255787]  0.859333  [0, 2.5362522048857983]   
2       NaN                     None       NaN                     None   

        mu3                     sig3  lambda  
0  1.770740  [0.4627010899611982, 0]    0.44  
1 -0.969678   [0, 2.904658290707424]    0.44  
2       NaN                     None    0.44  
iteration_new 18, shift_new 0.14381540100296578
        mu1                     sig1       mu2                     sig2  \
0 -0.440901  [1.1360515018131638, 0]  3.500992  [0.6790195634994599, 0]   
1 -2.190177  [0, 1.7267259912769877]  0.881124   [0, 2.529373965162247]   
2       NaN                     None       NaN                     None   

        mu3                     sig3    lambda  
0  1.715706  [0.4811017147700458, 0]  0.423333  
1 -1.061062  [0,